<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/data_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/nnUNet/nnunetv2')

In [ ]:
!nnUNetv2_plan_and_preprocess -d DATASET_ID --verify_dataset_integrity